You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: Harnessing the Power of Artificial Intelligence: Trends, Players, and News

Introduction:
Artificial Intelligence (AI) has revolutionized the way we interact with technology, making tasks easier and more efficient. In this blog article, we will explore the latest trends, key players, and noteworthy news in the field of AI, providing valuable insights for our readers.

Key Points:
1. Latest Trends in AI:
- Machine Learning and Deep Learning advancements
- Natural Language Processing (NLP) applications
- 

I now can give a great answer

Final Answer:
# Harnessing the Power of Artificial Intelligence: Trends, Players, and News

Artificial Intelligence (AI) has become an integral part of our daily lives, transforming the way we interact with technology and improving efficiency across various industries. In this blog post, we will delve into the latest trends, key players, and noteworthy news in the field of AI, offering valuable insights for tech enthusiasts, business professionals, and anyone interested in cutting-edge developments in AI.

## Latest Trends in AI

One of the most prominent trends in AI is the advancements in Machine Learning and Deep Learning technologies. These innovations have enabled AI systems to learn from data, recognize patterns, and make decisions with minimal human intervention. Additionally, Natural Language Processing (NLP) applications have revolutionized the way we communicate with AI-powered devices, enhancing user experiences and expanding the capabilities o

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Harnessing the Power of Artificial Intelligence: Trends, Players, and News

Artificial Intelligence (AI) has become an integral part of our daily lives, transforming the way we interact with technology and improving efficiency across various industries. In this blog post, we will delve into the latest trends, key players, and noteworthy news in the field of AI, offering valuable insights for tech enthusiasts, business professionals, and anyone interested in cutting-edge developments in AI.

## Latest Trends in AI

One of the most prominent trends in AI is the advancements in Machine Learning and Deep Learning technologies. These innovations have enabled AI systems to learn from data, recognize patterns, and make decisions with minimal human intervention. Additionally, Natural Language Processing (NLP) applications have revolutionized the way we communicate with AI-powered devices, enhancing user experiences and expanding the capabilities of virtual assistants.

AI-driven automation is another key trend, with industries leveraging AI technologies to streamline processes, increase productivity, and reduce operational costs. However, as AI continues to evolve, ethical considerations in AI development have become increasingly important. Ensuring transparency, accountability, and fairness in AI systems is crucial to building trust and mitigating potential risks associated with AI implementation.

## Key Players in the AI Industry

Several tech giants and innovative startups are leading the way in the AI industry, driving advancements and shaping the future of AI technology. Google's DeepMind, known for its contributions to AI research and development, continues to push the boundaries of AI capabilities. IBM Watson, Amazon Web Services (AWS), and Microsoft Azure are also key players in the AI space, offering AI-powered solutions and services to businesses worldwide. OpenAI, a nonprofit organization dedicated to advancing AI for the benefit of humanity, is another notable player in the industry.

## Noteworthy News in AI

In recent news, breakthroughs in AI healthcare solutions have shown promise in revolutionizing the healthcare industry, improving patient care, and enhancing medical diagnosis and treatment. AI applications in autonomous vehicles have also gained momentum, with companies investing in AI technologies to develop self-driving cars and enhance road safety. However, AI ethics controversies and discussions have sparked debates surrounding the responsible use of AI, privacy concerns, and bias in AI algorithms. On the cybersecurity front, AI advancements are being utilized to enhance threat detection, protect sensitive data, and strengthen cybersecurity defenses.

In conclusion, Artificial Intelligence continues to drive innovation, transform industries, and shape our future. By staying informed about the latest trends, key players, and noteworthy news in AI, we can harness the power of AI to drive positive change and create a more efficient and sustainable future.

To stay updated on the latest news and trends in Artificial Intelligence, subscribe to our newsletter for bi-weekly updates and exclusive content.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: Exploring the Latest Trends in Sustainable Fashion

Introduction:
- Brief overview of the current state of the fashion industry
- Importance of sustainable fashion
- Purpose of the article

Key Points:
1. Latest Trends in Sustainable Fashion
- Rise of eco-friendly materials such as organic cotton, recycled polyester, and Tencel
- Popularity of thrifting and second-hand shopping
- Emphasis on ethical production practices and fair labor conditions

2. Key Players in Sustainable Fashion
- B

I now can give a great answer

Final Answer:
# Exploring the Latest Trends in Sustainable Fashion

In today's fast-paced world, the fashion industry has come under increasing scrutiny for its environmental and social impact. The rise of fast fashion and mass production has highlighted the negative effects on the planet and garment workers, leading to a growing movement towards sustainable fashion. This shift towards sustainable practices emphasizes ethical production methods and the use of eco-friendly materials to minimize the industry's environmental footprint.

Sustainable fashion is not merely a passing trend; it is a crucial step towards securing a more sustainable future for both the fashion industry and the planet as a whole. By focusing on reducing waste, lowering carbon emissions, and ensuring fair labor conditions, sustainable fashion advocates for a more responsible and conscious approach to clothing production. This article aims to delve into the latest trends in sustainabl

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).